<a href="https://colab.research.google.com/github/m37335/kanagawa-exam/blob/master/part2_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install stanza

     |████████████████████████████████| 2.1MB 3.8MB/s 
     |████████████████████████████████| 3.3MB 47.0MB/s 
     |████████████████████████████████| 901kB 42.1MB/s 
     |████████████████████████████████| 286kB 4.0MB/s 


In [2]:
# pytorch
import torch
from transformers import BertTokenizer, BertForMaskedLM
# stanza
import stanza
stanza.download('en') # download English model

2021-05-04 11:36:10 INFO: Downloading default packages for language: en (English)...
2021-05-04 11:37:31 INFO: Finished downloading models and saved to /root/stanza_resources.


In [3]:
import pandas as pd
allsentenceInTextbook_df = pd.read_csv('https://raw.githubusercontent.com/m37335/kanagawa-exam/master/data/%20NH_textbook-sentence.csv')

In [4]:
allsentenceInTextbook_df

,id,textBook,grade,articleId,Page,sentenceId,sentence
0,0,NH,1,1,P4,1,"Good morning, Takashi."
1,1,NH,1,1,P4,2,"Good morning, Jill."
2,2,NH,1,1,P4,3,How are you?
3,3,NH,1,1,P4,4,"I'm fine, thank you."
4,4,NH,1,1,P4,5,And you?
...,...,...,...,...,...,...,...
1551,1551,NH,3,22,P111,38,"He used his photographs to share it with us,"
1552,1552,NH,3,22,P111,39,and to show us the importance of life on the E...
1553,1553,NH,3,22,P111,40,Michio’s own life was changed because of one p...
1554,1554,NH,3,22,P111,41,Perhaps his photographs will also change someo...


In [5]:
allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('”', '')
allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('“', '')
allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('"', '')
allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('[', '')
allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace(']', '')
allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('□', '')
allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('―', '')

#allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('?', ' ?')
#allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('!', ' !')
#allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('.', ' .')
#allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace(',', ' ,')
allsentenceInTextbook_df.sentence = allsentenceInTextbook_df.sentence.str.replace('\'\'', ' \'')

In [6]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma', use_gpu=True, tokenize_pretokenized=False)
stanza_token = []

for df_sentence in allsentenceInTextbook_df.sentence:
  doc = nlp(df_sentence)
  for sentence in doc.sentences:
    tmp_token = []
    for word in sentence.words:
      tmp_token.append(word.text)
    tmp_token.insert(0, "[CLS]")
    tmp_token.append("[SEP]")
    
    stanza_token.append(tmp_token)

#print(stanza_token)
stanza_token_df = pd.DataFrame(stanza_token)

2021-05-04 11:38:02 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-05-04 11:38:02 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-05-04 11:38:02 INFO: Use device: cpu
2021-05-04 11:38:02 INFO: Loading: tokenize
2021-05-04 11:38:02 INFO: Loading: pos
2021-05-04 11:38:02 INFO: Loading: lemma
2021-05-04 11:38:02 INFO: Done loading processors!


In [7]:
stanza_token_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,[CLS],Good,morning,",",Takashi,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,[CLS],Good,morning,",",Jill,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,[CLS],How,are,you,?,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,[CLS],I,'m,fine,",",thank,you,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,[CLS],And,you,?,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,[CLS],He,used,his,photographs,to,share,it,with,us,",",[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None
1552,[CLS],and,to,show,us,the,importance,of,life,on,the,Earth,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None
1553,[CLS],Michio,’s,own,life,was,changed,because,of,one,photograph,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None
1554,[CLS],Perhaps,his,photographs,will,also,change,someone,’s,life,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [8]:
df_concat = pd.concat([allsentenceInTextbook_df, stanza_token_df], axis=1)

In [61]:
from google.colab import files
filename =  "df_concat-check.csv"
df_concat.to_csv(filename, encoding = 'utf-8-sig') 
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
df_concat

,id,textBook,grade,articleId,Page,sentenceId,sentence,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,0,NH,1,1,P4,1,"Good morning, Takashi.",[CLS],Good,morning,",",Takashi,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1,NH,1,1,P4,2,"Good morning, Jill.",[CLS],Good,morning,",",Jill,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2,NH,1,1,P4,3,How are you?,[CLS],How,are,you,?,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,3,NH,1,1,P4,4,"I'm fine, thank you.",[CLS],I,'m,fine,",",thank,you,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,4,NH,1,1,P4,5,And you?,[CLS],And,you,?,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,1551,NH,3,22,P111,38,"He used his photographs to share it with us,",[CLS],He,used,his,photographs,to,share,it,with,us,",",[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None
1552,1552,NH,3,22,P111,39,and to show us the importance of life on the E...,[CLS],and,to,show,us,the,importance,of,life,on,the,Earth,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None
1553,1553,NH,3,22,P111,40,Michio’s own life was changed because of one p...,[CLS],Michio,’s,own,life,was,changed,because,of,one,photograph,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None
1554,1554,NH,3,22,P111,41,Perhaps his photographs will also change someo...,[CLS],Perhaps,his,photographs,will,also,change,someone,’s,life,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [10]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma', use_gpu=True, tokenize_pretokenized=False)
stanza_token_lemma = []

for df_sentence in allsentenceInTextbook_df.sentence:
  doc = nlp(df_sentence)
  for sentence in doc.sentences:
    tmp_token = []
    for word in sentence.words:
      tmp_token.append(word.lemma)
    tmp_token.insert(0, "[CLS]")
    tmp_token.append("[SEP]")
    
    stanza_token_lemma.append(tmp_token)

#print(stanza_token_lemma)
stanza_token_lemma_df = pd.DataFrame(stanza_token_lemma)

2021-05-04 11:39:21 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-05-04 11:39:21 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-05-04 11:39:21 INFO: Use device: cpu
2021-05-04 11:39:21 INFO: Loading: tokenize
2021-05-04 11:39:21 INFO: Loading: pos
2021-05-04 11:39:21 INFO: Loading: lemma
2021-05-04 11:39:21 INFO: Done loading processors!


In [11]:
stanza_token_lemma_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,[CLS],good,morning,",",Takashi,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,[CLS],good,morning,",",Jill,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,[CLS],how,be,you,?,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,[CLS],I,be,fine,",",thank,you,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,[CLS],and,you,?,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,[CLS],he,use,he,photograph,to,share,it,with,we,",",[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None
1552,[CLS],and,to,show,we,the,importance,of,life,on,the,Earth,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None
1553,[CLS],Michio,'s,own,life,be,change,because,of,one,photograph,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None
1554,[CLS],perhaps,he,photograph,will,also,change,someone,'s,life,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [12]:
lemma_df_concat = pd.concat([allsentenceInTextbook_df, stanza_token_lemma_df], axis=1)

In [13]:
lemma_df_concat

,id,textBook,grade,articleId,Page,sentenceId,sentence,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,0,NH,1,1,P4,1,"Good morning, Takashi.",[CLS],good,morning,",",Takashi,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1,NH,1,1,P4,2,"Good morning, Jill.",[CLS],good,morning,",",Jill,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2,NH,1,1,P4,3,How are you?,[CLS],how,be,you,?,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,3,NH,1,1,P4,4,"I'm fine, thank you.",[CLS],I,be,fine,",",thank,you,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,4,NH,1,1,P4,5,And you?,[CLS],and,you,?,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,1551,NH,3,22,P111,38,"He used his photographs to share it with us,",[CLS],he,use,he,photograph,to,share,it,with,we,",",[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None
1552,1552,NH,3,22,P111,39,and to show us the importance of life on the E...,[CLS],and,to,show,we,the,importance,of,life,on,the,Earth,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None
1553,1553,NH,3,22,P111,40,Michio’s own life was changed because of one p...,[CLS],Michio,'s,own,life,be,change,because,of,one,photograph,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None
1554,1554,NH,3,22,P111,41,Perhaps his photographs will also change someo...,[CLS],perhaps,he,photograph,will,also,change,someone,'s,life,.,[SEP],None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [14]:
f_wordlist_df = pd.read_csv('https://raw.githubusercontent.com/m37335/kanagawa-exam/master/data/f_wordlist.csv')

In [15]:
f_wordlist_df

,No,word,level,number of character
0,139,area,4,4
1,209,band,4,4
2,244,beef,4,4
3,260,belt,4,4
4,308,body,4,4
...,...,...,...,...
846,1138,grandmother,6,11
847,1374,interesting,6,11
848,2811,traditional,6,11
849,1375,international,6,13


In [16]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma', use_gpu=True, tokenize_pretokenized=False)
f_wordlist_lemma = []

for df_sentence in f_wordlist_df.word:
  doc = nlp(df_sentence)
  for sentence in doc.sentences:
    tmp_token = []
    for word in sentence.words:
      tmp_token.append(word.lemma)
    
    f_wordlist_lemma.append(tmp_token)

#print(stanza_token_lemma)
f_wordlist_lemma_df = pd.DataFrame(f_wordlist_lemma)

2021-05-04 11:41:46 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-05-04 11:41:46 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-05-04 11:41:46 INFO: Use device: cpu
2021-05-04 11:41:46 INFO: Loading: tokenize
2021-05-04 11:41:46 INFO: Loading: pos
2021-05-04 11:41:46 INFO: Loading: lemma
2021-05-04 11:41:46 INFO: Done loading processors!


In [17]:
f_wordlist_lemma_df

,0,1,2,3
0,area,None,None,None
1,band,None,None,None
2,beef,None,None,None
3,belt,None,None,None
4,body,None,None,None
...,...,...,...,...
846,grandmother,None,None,None
847,interesting,None,None,None
848,traditional,None,None,None
849,international,None,None,None


In [18]:
f_wordlist_lemma_df_concat = pd.concat([f_wordlist_df, f_wordlist_lemma_df], axis=1)

In [19]:
f_wordlist_lemma_df_concat

,No,word,level,number of character,0,1,2,3
0,139,area,4,4,area,None,None,None
1,209,band,4,4,band,None,None,None
2,244,beef,4,4,beef,None,None,None
3,260,belt,4,4,belt,None,None,None
4,308,body,4,4,body,None,None,None
...,...,...,...,...,...,...,...,...
846,1138,grandmother,6,11,grandmother,None,None,None
847,1374,interesting,6,11,interesting,None,None,None
848,2811,traditional,6,11,traditional,None,None,None
849,1375,international,6,13,international,None,None,None


In [20]:
words_list = stanza_token_lemma_df.values.tolist()
f_word_lemma_list = f_wordlist_lemma_df_concat[0].values.tolist()
f_word_list = f_wordlist_lemma_df_concat.word.values.tolist()


sentence_f_word_list = []

for f_word, f_word_lemma in zip(f_word_list, f_word_lemma_list):
  # print(f_word)
  # print(f_word_lemma)
  for cnt, list in enumerate(words_list, start=0):
    tmp_list = []
    try:
      tmp_list.append(cnt)
      tmp_list.append(f_word)
      tmp_list.append(f_word_lemma)
      tmp_list.append(list.index(f_word_lemma))
      sentence_f_word_list.append(tmp_list)
    except ValueError:
      pass

In [23]:
sentence_f_word_list

[[92, 'band', 'band', 7],
 [1255, 'band', 'band', 5],
 [1088, 'beef', 'beef', 5],
 [111, 'belt', 'belt', 5],
 [442, 'body', 'body', 13],
 [1273, 'body', 'body', 7],
 [1271, 'bomb', 'bomb', 10],
 [486, 'case', 'case', 7],
 [1303, 'case', 'case', 2],
 [1306, 'case', 'case', 2],
 [579, 'chef', 'chef', 6],
 [1468, 'dark', 'dark', 4],
 [235, 'fell', 'fall', 3],
 [1271, 'fell', 'fall', 11],
 [1276, 'fell', 'fall', 5],
 [1474, 'fell', 'fall', 4],
 [917, 'gift', 'gift', 15],
 [1170, 'goal', 'goal', 5],
 [1450, 'goal', 'goal', 11],
 [417, 'half', 'half', 7],
 [556, 'hall', 'hall', 5],
 [559, 'hall', 'hall', 5],
 [891, 'hate', 'hate', 2],
 [981, 'hero', 'hero', 2],
 [721, 'host', 'host', 4],
 [722, 'host', 'host', 3],
 [725, 'host', 'host', 5],
 [734, 'host', 'host', 2],
 [736, 'host', 'host', 3],
 [738, 'host', 'host', 7],
 [512, 'kept', 'keep', 3],
 [1314, 'kept', 'keep', 8],
 [1378, 'kept', 'keep', 7],
 [1425, 'kept', 'keep', 13],
 [1437, 'kept', 'keep', 5],
 [106, 'knew', 'know', 3],
 [196, 

In [25]:
sentence_f_word_df = pd.DataFrame(sentence_f_word_list,
                                  columns=['id', 'word', 'lemma', 'mask_id'])

In [53]:
mask_model_df = pd.DataFrame(columns=['id', 'textBook', 'grade', 'articleId', 'Page', 'sentenceId', 'sentence', 'mask_id'])

In [54]:
mask_model_df

,id,textBook,grade,articleId,Page,sentenceId,sentence,mask_id


In [76]:
id_list = sentence_f_word_df.id
maskId_list = sentence_f_word_df.mask_id

for id, maskId in zip(id_list, maskId_list):
  maskId_df = pd.Series([maskId], index=['mask_id'])

  mask_model_df = mask_model_df.append(allsentenceInTextbook_df.loc[allsentenceInTextbook_df['id'] == id])
  print(maskId)
  mask_model_df = pd.concat([mask_model_df, maskId_df], axis=1)
  # print(mask_model_df)


7


ValueError: ignored

In [63]:
mask_model_df.loc[mask_model_df['id'] == 92]


,id,textBook,grade,articleId,Page,sentenceId,sentence,mask_id
92,92,NH,1,6,P38,7,I play the guitar in a band.,7
92,92,NH,1,6,P38,7,I play the guitar in a band.,7
92,92,NH,1,6,P38,7,I play the guitar in a band.,7
92,92,NH,1,6,P38,7,I play the guitar in a band.,7
92,92,NH,1,6,P38,7,I play the guitar in a band.,7
92,92,NH,1,6,P38,7,I play the guitar in a band.,7


In [61]:
from google.colab import files
filename =  "mask_model_df_check.csv"
mask_model_df.to_csv(filename, encoding = 'utf-8-sig') 
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
allsentenceInTextbook_df

,id,textBook,grade,articleId,Page,sentenceId,sentence
0,0,NH,1,1,P4,1,"Good morning, Takashi."
1,1,NH,1,1,P4,2,"Good morning, Jill."
2,2,NH,1,1,P4,3,How are you?
3,3,NH,1,1,P4,4,"I'm fine, thank you."
4,4,NH,1,1,P4,5,And you?
...,...,...,...,...,...,...,...
1551,1551,NH,3,22,P111,38,"He used his photographs to share it with us,"
1552,1552,NH,3,22,P111,39,and to show us the importance of life on the E...
1553,1553,NH,3,22,P111,40,Michio’s own life was changed because of one p...
1554,1554,NH,3,22,P111,41,Perhaps his photographs will also change someo...


In [69]:
#df[df['col_A'].str.contains('A_1')]
# allsentenceInTextbook_df.loc[allsentenceInTextbook_df['id'].astype(str).str.contains('100')]
allsentenceInTextbook_df.loc[allsentenceInTextbook_df['id'] == 100]
#print(df[df['state'] == 'CA'])

TypeError: ignored

In [91]:
from google.colab import files
filename =  "f_wordlist_lemma_df_concatcheck.csv"
f_wordlist_lemma_df_concat.to_csv(filename, encoding = 'utf-8-sig') 
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
word_list = ['area', 'band', 'beef', 'belt', 'body']
